In [2]:
!git clone https://github.com/HKUDS/LightRAG.git

Cloning into 'LightRAG'...
remote: Enumerating objects: 1577, done.
remote: Counting objects: 100% (1033/1033), done.
remote: Compressing objects: 100% (366/366), done.
remote: Total 1577 (delta 847), reused 725 (delta 667), pack-reused 544 (from 1)
Receiving objects: 100% (1577/1577), 9.50 MiB | 12.58 MiB/s, done.
Resolving deltas: 100% (1046/1046), done.


In [3]:
%cd /content/LightRAG

/content/LightRAG


In [4]:
!pip install -e .

Obtaining file:///content/LightRAG
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.8 MB/s eta 0:00:00
 

`Now need to add the .txt file`

In [6]:
!pip install nest_asyncio

In [23]:
import os
import asyncio
from lightrag import LightRAG, QueryParam
from lightrag.utils import EmbeddingFunc
import numpy as np
from dotenv import load_dotenv
import logging
from openai import AzureOpenAI

import nest_asyncio
nest_asyncio.apply()

logging.basicConfig(level=logging.INFO)

load_dotenv()
from google.colab import userdata

# Retrieve environment variables
AZURE_OPENAI_API_VERSION = userdata.get('AZURE_OPENAI_API_VERSION')
AZURE_OPENAI_DEPLOYMENT = userdata.get("AZURE_OPENAI_DEPLOYMENT")
AZURE_OPENAI_API_KEY = userdata.get("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = userdata.get("AZURE_OPENAI_ENDPOINT")
AZURE_EMBEDDING_DEPLOYMENT = userdata.get("AZURE_EMBEDDING_DEPLOYMENT")
AZURE_EMBEDDING_API_VERSION = userdata.get("AZURE_EMBEDDING_API_VERSION")

# Ensure all variables are set
if not all([AZURE_OPENAI_API_VERSION, AZURE_OPENAI_DEPLOYMENT, AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, AZURE_EMBEDDING_DEPLOYMENT, AZURE_EMBEDDING_API_VERSION]):
    raise ValueError("One or more environment variables are not set.")


WORKING_DIR = "./dickens"

if os.path.exists(WORKING_DIR):
    import shutil
    shutil.rmtree(WORKING_DIR)

os.mkdir(WORKING_DIR)

async def llm_model_func(prompt, system_prompt=None, history_messages=[], keyword_extraction=False, **kwargs) -> str:
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
    )

    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    if history_messages:
        messages.extend(history_messages)
    messages.append({"role": "user", "content": prompt})

    try:
        chat_completion = client.chat.completions.create(
            model=AZURE_OPENAI_DEPLOYMENT,  # model = "deployment_name".
            messages=messages,
            temperature=kwargs.get("temperature", 0),
            top_p=kwargs.get("top_p", 1),
            n=kwargs.get("n", 1),
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        logging.error(f"Error in llm_model_func: {e}")
        raise

async def embedding_func(texts: list[str]) -> np.ndarray:
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_EMBEDDING_API_VERSION,
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
    )
    try:
        embedding = client.embeddings.create(model=AZURE_EMBEDDING_DEPLOYMENT, input=texts)
        embeddings = [item.embedding for item in embedding.data]
        return np.array(embeddings)
    except Exception as e:
        logging.error(f"Error in embedding_func: {e}")
        raise

async def test_funcs():
    try:
        result = await llm_model_func("How are you?")
        print("Resposta do llm_model_func: ", result)

        result = await embedding_func(["How are you?"])
        print("Resultado do embedding_func: ", result.shape)
        print("Dimensão da embedding: ", result.shape[1])
    except Exception as e:
        logging.error(f"Error in test_funcs: {e}")

asyncio.run(test_funcs())

embedding_dimension = 1536  # Update the embedding dimension to match the actual size

rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=llm_model_func,
    embedding_func=EmbeddingFunc(
        embedding_dim=embedding_dimension,
        max_token_size=8192,
        func=embedding_func,
    ),
)

book = open("./horror_story.txt", encoding="utf-8")

rag.insert(book.read())

query_text = "What are the main themes?"

print("Result (Naive):")
print(rag.query(query_text, param=QueryParam(mode="naive")))

print("\nResult (Local):")
print(rag.query(query_text, param=QueryParam(mode="local")))

print("\nResult (Global):")
print(rag.query(query_text, param=QueryParam(mode="global")))

print("\nResult (Hybrid):")
print(rag.query(query_text, param=QueryParam(mode="hybrid")))


Resposta do llm_model_func:  I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?
Resultado do embedding_func:  (1, 1536)
Dimensão da embedding:  1536


Extracting entities from chunks:   0%|          | 0/1 [00:00<?, ?chunk/s]

Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.14s/batch]


Result (Naive):
### Main Themes in the Story

The story of Alex, Ben, Clara, Diana, and Ethan exploring the haunted mansion on a dark and stormy night touches on several key themes. Here are the main themes explored in the narrative:

#### 1. **Fear and Courage**
The theme of fear is prevalent throughout the story, starting from the moment the friends decide to explore the mansion. Clara's initial hesitation and trembling voice highlight the fear that grips the group. However, the story also showcases courage, particularly when Clara remembers the lore about spirits being bound to objects and uses the locket to confront the ghost. This act of bravery ultimately saves them.

#### 2. **Friendship and Unity**
The friends' unity is a significant theme as they face the unknown together. Despite their fear, they stick together, supporting each other through the terrifying experience. Their collective effort to explore the mansion and their shared terror when confronted by the spirit undersco

```json
{
  "high_level_keywords": ["Main themes"],
  "low_level_keywords": []
}
```
Sorry, I'm not able to provide an answer to that question.

Result (Global):
```json
{
  "high_level_keywords": ["Main themes"],
  "low_level_keywords": []
}
```
### Main Themes in the Story

The narrative of the friends exploring the haunted mansion is rich with several prominent themes that drive the plot and character development. Here are the main themes:

#### 1. **Curiosity and Adventure**
The story begins with the friends' decision to explore the abandoned mansion, driven by curiosity and a sense of adventure. This theme is evident in their collective decision to investigate the mansion despite its reputation for being haunted. The exploration itself sets the stage for the unfolding events and highlights the human desire to seek out the unknown, even at the risk of encountering danger.

#### 2. **Fear and Bravery**
Fear is a pervasive theme throughout the story, as the friends experience various

```json
{
  "high_level_keywords": ["Main themes"],
  "low_level_keywords": []
}
```
Sorry, I'm not able to provide an answer to that question.
